In [ ]:
import numpy as np
import healpy as hp
from healpy import projview
import emcee
from tqdm import tqdm
import matplotlib.pyplot as plt
import pyccl as ccl
import pandas as pd
from astropy.coordinates import SkyCoord
import os

# ==============================================
# 1. LEITURA E PREPARAÇÃO DOS DADOS OBSERVACIONAIS
# ==============================================

# Parâmetros fixos
H0 = 67.27  # Valor fixo do Hubble
h = H0 / 100
Omega_b = 0.0494
sigma8 = 0.8120
n_s = 0.9646

# Carregar o catálogo Pantheon+
data = pd.read_csv('/home/sofia/Documentos/Pantheon/Pantheon+SH0ES.csv', sep=' ')
cove = np.genfromtxt('/home/sofia/Documentos/Pantheon/Pantheon+SH0ES_STAT+SYS.cov', skip_header=1)
full_cov = np.reshape(cove, (1701, 1701))

# Adicionar índice às supernovas
data['index'] = np.arange(len(data))

# Filtrar por redshift (exemplo: 0.1 < z < 1.0)
z_min, z_max = 0.15, 0.5
data_filtered = data[(data['zCMB'] >= z_min) & (data['zCMB'] <= z_max)]

In [ ]:
# ==============================================
# 2. DIVISÃO EM PIXELS HEALPix (nside=2)
# ==============================================

nside = 2  
npix = hp.nside2npix(nside)
raio = 90  # graus

# Obter coordenadas dos centros dos pixels
lon, lat = hp.pix2ang(nside, range(npix), lonlat=True)
galactic_coords = SkyCoord(l=lon, b=lat, frame='galactic', unit='deg')
equatorial_coords = galactic_coords.icrs
ra_pixel = equatorial_coords.ra.deg
dec_pixel = equatorial_coords.dec.deg

# Criar diretório para salvar os pixels
output_dir = '/home/sofia/Documentos/Pantheon/Atividade 4/Hemisferios_MCMC'  
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# ==============================================
# 3. ASSOCIAR SUPERNOVAS A CADA PIXEL
# ==============================================

# Converter coordenadas das supernovas para radianos
ra_data = np.radians(data_filtered['RA'].values)
dec_data = np.radians(data_filtered['DEC'].values)

# Para cada pixel, encontrar supernovas dentro do raio
pixel_data_list = []
for i in tqdm(range(npix), desc="Processando pixels"):
    # Coordenadas do centro do pixel
    ra_centro = np.radians(ra_pixel[i])
    dec_centro = np.radians(dec_pixel[i])
    
    # Fórmula de Haversine
    delta_ra = ra_data - ra_centro
    delta_dec = dec_data - dec_centro
    a = np.sin(delta_dec/2)**2 + np.cos(dec_centro)*np.cos(dec_data)*np.sin(delta_ra/2)**2
    distancia = np.degrees(2 * np.arcsin(np.sqrt(a)))
    
    # Selecionar supernovas dentro do raio
    mask = distancia <= raio
    pixel_data = data_filtered[mask].copy()
    
    # Salvar dados do pixel
    pixel_file = os.path.join(output_dir, f'pixel_{i:03d}.csv')
    pixel_data.to_csv(pixel_file, index=False)
    pixel_data_list.append(pixel_data)

In [ ]:
# ==============================================
# 4. FUNÇÕES PARA O AJUSTE MCMC
# ==============================================

def mu_theory(z, Omega_m):
    """Calcula a magnitude de distância teórica usando PyCCL"""
    cosmo = ccl.Cosmology(
        Omega_c=Omega_m - Omega_b,
        Omega_b=Omega_b,
        h=h,
        sigma8=sigma8,
        n_s=n_s)
    a = 1/(1+z)
    return ccl.background.distance_modulus(cosmo, a)

def log_probability(theta, z_obs, mu_obs, inv_cov):
    """Função de log-probabilidade para o MCMC"""
    Omega_m = theta[0]
    
    # Prior uniforme
    if not 0.1 < Omega_m < 0.5:
        return -np.inf
    
    # Calcular predição teórica
    mu_theo = mu_theory(z_obs, Omega_m)
    
    # Calcular chi²
    delta = mu_obs - mu_theo
    chi2 = np.dot(delta, np.dot(inv_cov, delta))
    
    return -0.5 * chi2

In [ ]:
# ==============================================
# 5. EXECUÇÃO DO MCMC PARA CADA PIXEL 
# ==============================================

nwalkers = 4  # Número maior de caminhantes
nsteps = 100   # Mais passos para convergência
burnin = 20    # Burn-in mais generoso

# Inicializar arrays para armazenar resultados
Om_map_mcmc = np.full(npix, np.nan)  # Valores medianos de Omega_m para cada pixel
Om_error_map = np.full(npix, np.nan)  # Erros padrão para cada pixel

print(f"> Pixel {i}: N_sne={len(pixel_data)}, z_mean={np.mean(z_obs):.2f}, mu_mean={np.mean(mu_obs):.2f}")

for i in tqdm(range(npix), desc="Ajustando pixels com MCMC"):
    try:
        print(f"\n=== Processando pixel {i} ===")
        pixel_file = os.path.join(output_dir, f'pixel_{i:03d}.csv')
        
        # Verificação rigorosa do arquivo
        if not os.path.exists(pixel_file):
            print(f"> Arquivo do pixel {i} não encontrado!")
            continue
            
        if os.path.getsize(pixel_file) == 0:
            print(f"> Arquivo do pixel {i} vazio!")
            continue
            
        pixel_data = pd.read_csv(pixel_file)
        print(f"> Número de supernovas: {len(pixel_data)}")
        
        # Verificação dos dados básicos
        if len(pixel_data) < 2:  # Reduzido para debug
            print(f"> Muito poucas supernovas ({len(pixel_data)})")
            continue
            
        z_obs = pixel_data['zCMB'].values
        mu_obs = pixel_data['MU_SH0ES'].values
        indices = pixel_data['index'].values
        
        print(f"> z range: [{z_obs.min():.3f}, {z_obs.max():.3f}]")
        print(f"> mu range: [{mu_obs.min():.3f}, {mu_obs.max():.3f}]")
        
        # Verificação da matriz de covariância
        try:
            cov_sub = full_cov[np.ix_(indices, indices)]
            print(f"> Dimensão da matriz de cov: {cov_sub.shape}")
            
            # Verificação de valores extremos
            if np.any(np.isnan(cov_sub)):
                print("> Matriz de cov contém NaN!")
                continue
                
            if np.all(cov_sub == 0):
                print("> Matriz de cov é toda zero!")
                continue
                
            inv_cov_sub = np.linalg.pinv(cov_sub)
            print("> Matriz inversa calculada com sucesso")
            
        except Exception as e:
            print(f"> Erro na matriz de cov: {str(e)}")
            continue
            
        # Configuração inicial do MCMC
        initial_guess = 0.3
        pos = initial_guess + 0.01 * np.random.randn(nwalkers, 1)
        
        # Função de verificação da log-probabilidade
        test_theta = np.array([0.3])
        test_logprob = log_probability(test_theta, z_obs, mu_obs, inv_cov_sub)
        print(f"> Teste logprob (Ωm=0.3): {test_logprob:.2f}")
        
        if not np.isfinite(test_logprob):
            print("> Logprob retornou infinito para valor inicial!")
            continue
            
        # Execução do MCMC com verificação em tempo real
        sampler = emcee.EnsembleSampler(
            nwalkers, 1, log_probability,
            args=(z_obs, mu_obs, inv_cov_sub))
        
        print("> Iniciando MCMC...")
        state = sampler.run_mcmc(pos, nsteps, progress=True)
        
        # Verificação das cadeias
        samples = sampler.get_chain(discard=burnin, flat=True)
        print(f"> Número de amostras válidas: {len(samples)}")
        
        if len(samples) == 0:
            print("> Nenhuma amostra válida gerada!")
            continue
            
        # Cálculo robusto das estatísticas
        Om_map_mcmc[i] = np.median(samples) if len(samples) > 0 else np.nan
        Om_error_map[i] = np.std(samples) if len(samples) > 1 else np.nan
        
        print(f"> Resultado: Ωm = {Om_map_mcmc[i]:.3f} ± {Om_error_map[i]:.3f}")
        
        # Plot rápido para diagnóstico (opcional)
        plt.figure()
        plt.plot(sampler.get_chain()[:, :, 0].T, alpha=0.3)
        plt.title(f"Pixel {i} - Cadeias MCMC")
        plt.ylabel("Ωm")
        plt.savefig(os.path.join(output_dir, f'chain_pixel_{i:03d}.png'))
        plt.close()
        
    except Exception as e:
        print(f"> ERRO GRAVE: {str(e)}")
        continue

In [ ]:
# ==============================================
# 6. VISUALIZAÇÃO GRÁFICA - Ωm e ERROS
# ==============================================

# Configurações comuns
fontsize = 14
cmap_omega = 'jet'       # Paleta para Ωm
cmap_error = 'viridis'   # Paleta para erros
bg_color = 'white'       # Cor de fundo
dpi = 600                # Resolução

# Função para plotar mapas padronizados
def plot_healpix_map(map_data, title, filename, cmap, vmin=None, vmax=None):
    plt.figure(facecolor=bg_color, figsize=(10, 7))
    
    # Cria uma cópia dos dados para evitar modificações no array original
    plot_data = np.array(map_data.copy())
    
    # Configuração da normalização
    if vmin is not None and vmax is not None:
        norm = None  # Usaremos os limites diretamente
    else:
        norm = None
        
    # Visualização com tratamento robusto
    try:
        hp.projview(
            plot_data,
            coord=["G"],
            cmap=cmap,
            title=title,
            graticule=True,
            graticule_labels=True,
            longitude_grid_spacing=30,
            projection_type="mollweide",
            fontsize={'title': fontsize, 'label': fontsize-2, 'tick': fontsize-4},
            min=vmin,
            max=vmax,
            unit="Ωm",
            hold=True
        )
    except Exception as e:
        print(f"Erro na visualização: {str(e)}")
        # Fallback para mollview se projview falhar
        hp.mollview(
            plot_data,
            title=title,
            cmap=cmap,
            min=vmin,
            max=vmax,
            unit="Ωm"
        )
        hp.graticule()
    
    # Ajustes finais
    ax = plt.gca()
    ax.set_facecolor(bg_color)
    
    plt.savefig(
        os.path.join(output_dir, filename),
        dpi=dpi,
        bbox_inches='tight',
        transparent=False,
        facecolor=bg_color
    )
    plt.close()

# Plotando Ωm com limites ajustados para seus valores
plot_healpix_map(
    Om_map_mcmc,
    title=f"$\Omega_m$-map (MCMC) para ${z_min}<z<{z_max}$",
    filename='Om_map_mcmc.png',
    cmap=cmap_omega,
    vmin=0.48,  # Ajustado para seus valores reais
    vmax=0.50
)

# Plotando erros
plot_healpix_map(
    Om_error_map,
    title=f"Erros de $\Omega_m$ para ${z_min}<z<{z_max}$",
    filename='Om_error_map.png',
    cmap=cmap_error
)